# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [17]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download(['punkt', 'wordnet'])
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
import pickle
import joblib

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterETL.db')
df = pd.read_sql_table(table_name= 'CleanedMessages', con = engine)
X = df['message']
y = df.drop(['id', 'genre', 'original', 'message'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    text="".join([c for c in text if c not in string.punctuation])
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        if tok not in stopwords.words('english'):
            clean_tok = lemmatizer.lemmatize(tok).lower().strip()
            clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = pipeline = Pipeline(
    [
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ]
)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test)

In [7]:
for p, column in enumerate(y_test.columns):
    report = classification_report(y_test.iloc[:, p], pd.DataFrame(y_pred).iloc[:, p])
    print('Column:'+column)
    print(report)

Column:related
             precision    recall  f1-score   support

          0       0.35      0.15      0.21      1583
          1       0.76      0.91      0.83      4877
          2       1.00      0.02      0.04        51

avg / total       0.66      0.72      0.67      6511

Column:request
             precision    recall  f1-score   support

          0       0.83      0.98      0.90      5365
          1       0.47      0.08      0.14      1146

avg / total       0.77      0.82      0.77      6511

Column:offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6477
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6511

Column:aid_related
             precision    recall  f1-score   support

          0       0.59      0.79      0.68      3802
          1       0.44      0.23      0.31      2709

avg / total       0.53      0.56      0.52      6511

Column:medical_help
     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
#estimator.get_params().keys()

In [9]:
parameters = {
        'clf__estimator__n_estimators': [50, 100, 200]
    }
cv = GridSearchCV(pipeline, param_grid=parameters , verbose=10, n_jobs=4)

In [10]:
cv.fit(X_train, y_train)
print("\nBest Parameters:", cv.best_params_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] clf__estimator__n_estimators=50 .................................
[CV] clf__estimator__n_estimators=50 .................................
[CV] clf__estimator__n_estimators=50 .................................
[CV] clf__estimator__n_estimators=100 ................................
[CV]  clf__estimator__n_estimators=50, score=0.15468509984639017, total=26.6min
[CV] clf__estimator__n_estimators=100 ................................
[CV]  clf__estimator__n_estimators=50, score=0.15527568729841806, total=26.7min
[CV] clf__estimator__n_estimators=100 ................................
[CV]  clf__estimator__n_estimators=50, score=0.1552995391705069, total=26.9min
[CV] clf__estimator__n_estimators=200 ................................


[Parallel(n_jobs=4)]: Done   3 out of   9 | elapsed: 33.2min remaining: 66.4min


[CV]  clf__estimator__n_estimators=100, score=0.15696513592382122, total=47.3min
[CV] clf__estimator__n_estimators=200 ................................


[Parallel(n_jobs=4)]: Done   4 out of   9 | elapsed: 55.6min remaining: 69.5min


[CV]  clf__estimator__n_estimators=100, score=0.1542242703533026, total=46.7min
[CV] clf__estimator__n_estimators=200 ................................


...

[CV]  clf__estimator__n_estimators=200, score=0.15711872216249423, total=79.2min


[Parallel(n_jobs=4)]: Done   7 out of   9 | elapsed: 121.5min remaining: 34.7min


[CV]  clf__estimator__n_estimators=200, score=0.1565284178187404, total=71.3min
[CV]  clf__estimator__n_estimators=200, score=0.15714285714285714, total=50.3min


[Parallel(n_jobs=4)]: Done   9 out of   9 | elapsed: 141.3min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   9 out of   9 | elapsed: 141.3min finished



Best Parameters: {'clf__estimator__n_estimators': 200}


In [11]:
y_pred_best = pd.DataFrame(cv.predict(X_test))

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
for p, column in enumerate(y_test.columns):
    report = classification_report(y_test.iloc[:, p], pd.DataFrame(y_pred).iloc[:, p])
    print('Column:'+ column)
    print(report)

Column:related
             precision    recall  f1-score   support

          0       0.35      0.15      0.21      1583
          1       0.76      0.91      0.83      4877
          2       1.00      0.02      0.04        51

avg / total       0.66      0.72      0.67      6511

Column:request
             precision    recall  f1-score   support

          0       0.83      0.98      0.90      5365
          1       0.47      0.08      0.14      1146

avg / total       0.77      0.82      0.77      6511

Column:offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6477
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6511

Column:aid_related
             precision    recall  f1-score   support

          0       0.59      0.79      0.68      3802
          1       0.44      0.23      0.31      2709

avg / total       0.53      0.56      0.52      6511

Column:medical_help
     

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [15]:
filename = 'disaster_response_ML_model.sav'
pickle.dump(cv, open(filename, 'wb'))

In [18]:
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, y_test)
print(result)

0.150053755184


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.